# NBA Season Stats Management

In [1]:
import asyncio
import pandas as pd
from pony.orm import *

## Create the database connection

In [99]:
db = Database()
# PostgreSQL
db.bind(provider='postgres', user='postgres', password='05041997', host='localhost', database='nba')

## Map All the entity of logical model through PonyORM

In [3]:
class Team(db.Entity):
    id = PrimaryKey(int, auto=True)
    name = Required(str, 25, unique=True, nullable=False)
    abbr = Required(str, 5, unique=False, nullable=False)
    location = Optional(str, 25, nullable=True)
    player = Set(lambda : Player)
    stats = Optional(lambda : TeamStats, cascade_delete=True)
    coach = Optional(lambda : Coach)


class Player(db.Entity):
    name = Required(str, 50)
    position = Optional(str, 30, nullable=True)
    age = Optional(int, size=8, nullable=True)
    team = Optional(Team)
    stats = Optional(lambda : PlayerStats, cascade_delete=True)

class Coach(db.Entity):
    name = PrimaryKey(str, 50)
    team = Optional(Team)
    stats = Optional(lambda : CoachStats, cascade_delete=True)


class TeamStats(db.Entity):
    id = PrimaryKey(int, size=8, auto=True)
    G = Required(int)
    MP = Required(int)
    FG = Required(int)
    FGA = Required(int)
    FGP = Required(float)
    ThreeP = Required(int)
    ThreePA = Required(int)
    ThreePP = Required(float)
    TwoP = Required(int)
    TwoPA = Required(int)
    TwoPP = Required(float)
    FT = Required(int)
    FTA = Required(int)
    FTP = Optional(float)
    ORB = Required(int)
    DRB = Required(int)
    TRB = Required(int)
    AST = Required(int)
    STL = Required(int)
    BLK = Required(int)
    TOV = Required(int)
    PF = Required(int)
    PTS = Required(int)
    team = Optional(Team, cascade_delete=True)


class PlayerStats(db.Entity):
    id = PrimaryKey(int, auto=True)
    GMS = Required(int)
    GStart = Required(int)
    MP = Required(int)
    FG = Required(int)
    FGA = Required(int)
    FGP = Required(float)
    ThreeP = Required(int)
    ThreePA = Required(int)
    ThreePP = Required(float)
    TwoP = Required(float)
    TwoPA = Required(float)
    TwoPP = Required(float)
    eFGP = Required(float)
    FT = Required(float)
    FTA = Required(float)
    FTP = Optional(float)
    ORB = Required(float)
    DRB = Required(float)
    TRB = Required(float)
    AST = Required(float)
    STL = Required(float)
    BLK = Required(float)
    TOV = Required(float)
    PF = Required(float)
    PTS = Required(float)
    player = Optional(Player)

class CoachStats(db.Entity):
    id = PrimaryKey(int, auto=True)
    SeasG = Required(int)
    SeasW = Required(int)
    SeasL = Optional(float)
    FranG = Required(int)
    FranW = Required(int)
    FranL = Required(int)
    CareW = Required(int)
    CareL = Required(int)
    CareWP = Required(float)
    POSeasG = Required(float)
    POSeasW = Required(float)
    POSeasL = Required(float)
    POFranW = Required(float)
    POFranG = Required(float)
    POFranL = Required(float)
    POCareG = Required(float)
    POCareW = Required(float)
    POCareL = Required(float)
    coach = Optional(Coach)

db.generate_mapping(create_tables=True)

## Let's populate the database schema just created

In [4]:
full = pd.read_csv("../CSV_Files/full_team_coach_data.csv", index_col=0)
playerdf = pd.read_csv("../CSV_Files/full_player.csv")

In [5]:
# Def used to insert a team from processed data into database
def insert_team(data):
    team = Team(id=data['TeamID'], name=data['TeamName'], abbr=data['TeamAbbr'], location=data['Location'])
    return team

# def used to insert team stats into a database
def insert_team_stats(data, team: Team, idx: int):
    stats = TeamStats(
        id = idx,
        G = data['Gms'],
        MP = data['MP'],
        FG = data['FG'],
        FGA = data['FGA'],
        FGP = data['FGP'],
        ThreeP = data['ThreeP'],
        ThreePA = data['ThreePA'],
        ThreePP = data['ThreePP'],
        TwoP = data['TwoP'],
        TwoPA = data['TwoPA'],
        TwoPP = data['TwoP'],
        FT = data['FT'],
        FTA = data['FTA'],
        FTP = data['FTP'],
        ORB = data['ORB'],
        DRB = data['DRB'],
        TRB = data['TRB'],
        AST = data['AST'],
        STL = data['STL'],
        BLK = data['BLK'],
        TOV = data['TOV'],
        PF = data['PF'],
        PTS = data['PTS'],
        team = team
    )
    return stats

# def used to insert a coach into a database
def insert_coach(data, team: Team):
    coach = Coach(
        name = data['Name'],
        team = team
    )
    return coach

# def used to insert coach stats into a database e set relationship with a coach
def insert_coach_stats(data, coach: Coach, idx):
    stats = CoachStats(
    id = idx,
    SeasG = data['SeasG'],
    SeasW = data['SeasW'],
    SeasL = data['SeasL'],
    FranG = data['FranG'],
    FranW = data['FranW'],
    FranL = data['FranL'],
    CareW = data['CareW'],
    CareL = data['CareL'],
    CareWP = data['CareWP'],
    POSeasG = data['POSeasG'],
    POSeasW = data['POSeasW'],
    POSeasL = data['POSeasL'],
    POFranW = data['POFranW'],
    POFranG = data['POFranG'],
    POFranL = data['POFranL'],
    POCareG = data['POCareG'],
    POCareW = data['POCareW'],
    POCareL = data['POCareL'],
    coach = coach
    )
    return stats


In [6]:
idx = 0
for index, row in full.iterrows():
    team = insert_team(row)
    coach = insert_coach(row, team)
    team_stats = insert_team_stats(row, team, idx)
    coach_stats = insert_coach_stats(row, coach, idx)
    team.coach = coach
    team.stats = team_stats
    coach.stats = coach_stats
    commit()
    idx += 1

In [7]:
def insert_player(data, idx):
    player = Player(
        id = idx,
        name = data['Name'],
        position =  data['Pos'],
        age = data['Age'],
        team = Team.select(lambda tm : tm.id == data['TeamID']).get()
    )
    return player

def insert_player_stats(data, idx, player):
    stats = PlayerStats(
        id = idx,
        GMS = data['Gms'],
        GStart = data['Gstart'],
        MP = data['MP'],
        FG = data['FG'],
        FGA = data['FGA'],
        FGP = data['FGP'],
        ThreeP = data['ThreeP'],
        ThreePA = data['ThreePA'],
        ThreePP = data['ThreePP'],
        TwoP = data['TwoP'],
        TwoPA = data['TwoPA'],
        TwoPP = data['TwoPP'],
        eFGP = data['eFGP'],
        FT = data['FT'],
        FTA = data['FTA'],
        FTP = data['FTP'],
        ORB = data['ORB'],
        DRB = data['DRB'],
        TRB = data['TRB'],
        AST = data['AST'],
        STL = data['STL'],
        BLK = data['BLK'],
        TOV = data['TOV'],
        PF = data['PF'],
        PTS = data['PTS'],
        player = player
    )
    return stats


In [8]:
idx = 0
for index, row in playerdf.iterrows():
    player = insert_player(row, idx)
    stats = insert_player_stats(row, idx, player)
    player.stats = stats
    team = Team.select(lambda tm: tm.id == int(row['TeamID'])).get()
    team.player.add(player)
    commit()
    idx += 1

## CRUD Operation on data

In [180]:
@db_session
def get_all_team():
    return Team.select()

@db_session
def get_team(idx:int):
    return Team.select(lambda t: t.id == idx).get()

@db_session
def get_team_stats(idx:int):
    return TeamStats.select(lambda ts: ts.id == idx).get()

@db_session
def get_coach(idx:int):
    return Coach.select(lambda c: c.team.id == idx).get()

@db_session
def get_coach_stats(idx:str):
    return CoachStats.select(lambda cs: cs.coach.name == idx).get()

@db_session
def get_players(idx:int):
    return Player.select(lambda p: p.team.id == idx)

@db_session
def get_player_stats(idx:int):
    return PlayerStats.select(lambda ps: ps.player.id == idx).get()

In [ ]:
## Implementation of FastAPI to create API of nba data

In [188]:
from fastapi import FastAPI
from pydantic import BaseModel
from loguru import logger
from pony.orm.serialization import to_dict

app = FastAPI()

class UserRequestIn(BaseModel):
    text: str

@app.get("/team/{idx}/coach")
@db_session
def coach_get(idx:int):
    c = get_coach(idx)
    return {
        "name" : c.name,
        "stats" : f'http://localhost:3000/team/{idx}/{c.name}/stats'
    }

@app.get("/team/{coach}/stats")
@db_session
def coach_stats_get(coach:str):
    c = get_coach_stats(coach)
    data = {}
    for key,value in c.to_dict().items():
        if key == "coach" or key == "id":
            continue
        data[key] = value
    return data

@app.get('/team/{idx}/players')
@db_session
def players_get(idx:int):
    players = get_players(idx)
    all_players = []
    for el in players:
        data = {
                'id' : el.id,
                'name': el.name,
                'position': el.position,
                'age': el.age,
                'team': f'http://localhost:3000/team/{el.team.id}',
                'stats': f'http://localhost:3000/player/{el.id}/stats'
        }
        all_players.append(data)
    return all_players

@app.get('/player/{idx}/stats')
@db_session
def players_stats_get(idx:int):
    pstat = get_player_stats(idx)
    data = {}
    for key,value in pstat.to_dict().items():
        if key == 'player' : continue
        data[key] = value

    return data

@app.get('/team/{idx}')
@db_session
def team_get(idx:int):
    el = get_team(idx)
    data = {
                'id' : el.id,
                'name': el.name,
                'abbr': el.abbr,
                'location': el.location,
                'stats': f'http://localhost:3000/team/{el.id}/stats',
                'coach': f'http://localhost:3000/team/{el.id}/coach',
                'players': f'http://localhost:3000/team/{el.id}/players'}

    return data

@app.get('/team/{teamid}/stats')
@db_session
def team_stats_get(teamid:int):
    stats = get_team_stats(12)
    return { "id" : stats.id}


@app.get('/')
@db_session
def root():
    teams = get_all_team()
    all_teams = []
    for el in teams:
        data = {'name': el.name,
                'abbr': el.abbr,
                'location': el.location,
                'stats': f'http://localhost:3000/team/{el.id}/stats',
                'coach': f'http://localhost:3000/team/{el.id}/coach',
                'players': f'http://localhost:3000/team/{el.id}/players'}
        all_teams.append(data)
    return all_teams

In [189]:
from pyngrok import ngrok

ngrok_tunnel = ngrok.connect(3000)

ngrok_tunnel

<NgrokTunnel: "http://b31f-185-234-173-130.ngrok.io" -> "http://localhost:3000">

In [190]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run(app, port=3000)

INFO:     Started server process [53461]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:3000 (Press CTRL+C to quit)


INFO:     127.0.0.1:64117 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:64117 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:64117 - "GET /team/12/stats HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/Users/biagiolicari/miniforge3/envs/workspace/lib/python3.10/site-packages/uvicorn/protocols/http/h11_impl.py", line 364, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/Users/biagiolicari/miniforge3/envs/workspace/lib/python3.10/site-packages/uvicorn/middleware/proxy_headers.py", line 75, in __call__
    return await self.app(scope, receive, send)
  File "/Users/biagiolicari/miniforge3/envs/workspace/lib/python3.10/site-packages/fastapi/applications.py", line 212, in __call__
    await super().__call__(scope, receive, send)
  File "/Users/biagiolicari/miniforge3/envs/workspace/lib/python3.10/site-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/Users/biagiolicari/miniforge3/envs/workspace/lib/python3.10/site-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc
  File "/Users